In [1]:
from pathlib import Path
import cv2
import sys
import numpy as np
from skimage import feature, measure
import livecell_tracker
from livecell_tracker.core import datasets
from livecell_tracker.core.datasets import LiveCellImageDataset
import livecell_tracker.segment
from livecell_tracker import core
import livecell_tracker.core.utils
from tqdm import tqdm
import json
from livecell_tracker.core import (
    SingleCellTrajectory,
    SingleCellStatic,
    SingleCellTrajectoryCollection,
)
import json

seg_dir_path = r"""../datasets/test_data/track_sort/seg_output_3_imgs_day0_group1"""
label_seg_imgs = core.datasets.LiveCellImageDataset(seg_dir_path, ext="png")

label_seg_imgs = LiveCellImageDataset(seg_dir_path, ext="png", num_imgs=3)
dataset_dir_path = Path(
    r"../cxa-data/june_2022_data_8bit_png/restart_day0_Group 1_wellA1_RI_MIP_stitched"
)

raw_imgs = LiveCellImageDataset(dataset_dir_path, ext="png", num_imgs=3)
segmentation_result_json_path = r"../datasets/test_data/track_sort/seg_output_3_imgs_day0_group1/segmentation_results.json"
segmentation_results = json.load(open(segmentation_result_json_path, "r"))
segmentation_results;

In [2]:
import napari
from skimage import data
# viewer = napari.view_image(raw_imgs[0], name='image')
viewer = napari.view_image(raw_imgs.to_dask(), name='image', cache=False)
# all_imgs = LiveCellImageDataset(dataset_dir_path, ext="png", num_imgs=none)

# viewer = napari.view_image(all_imgs.to_dask(), name='all-image', cache=False)

In [4]:
traj_collection_json_path = "../datasets/test_data/traj_analysis/track_singleCellTrajectoryCollection.json"
traj_collection_json = json.load(open(traj_collection_json_path, "r"))
trajectory_collection = SingleCellTrajectoryCollection().load_from_json_dict(traj_collection_json)

In [25]:
traj_1 = trajectory_collection[1]
rec_shapes = traj_1.get_sc_napari_shapes()
for i, rec in rec_shapes.items():
    rec_shapes[i] = np.array(rec) + 100 * i

shapes_layer = viewer.add_shapes([rec_shapes[0]], name="traj1")
# for traj in trajectory_collection:
#     viewer.add_shapes(traj.get_sc_napari_shapes(), opacity=0.1, name="traj")


In [23]:
shapes_layer.data = []

In [22]:
def set_shapes_features(shapes_layer, step2shapes, step):
    # step is a 4D coordinate with the current slider position for each dim
    column = step[0]  # grab the leading ("time") coordinate
    new_data = []
    print(step)
    if step in step2shapes:
        new_data = step2shapes[step]
    shapes_layer.data = new_data
    shapes_layer.face_color = 'value'  # force features refresh


viewer.dims.events.current_step.connect(
        lambda event: set_shapes_features(shapes_layer, traj_1.get_sc_napari_shapes(), event.value)
        )


<function __main__.<lambda>(event)>

(1, 1204, 1204)
(1, 1204, 1204)
(1, 1204, 1204)
(0, 1204, 1204)
(0, 1204, 1204)
(0, 1204, 1204)
(1, 1204, 1204)
(1, 1204, 1204)
(1, 1204, 1204)
(0, 1204, 1204)
(0, 1204, 1204)
(0, 1204, 1204)
(1, 1204, 1204)
(1, 1204, 1204)
(1, 1204, 1204)
(0, 1204, 1204)
(0, 1204, 1204)
(0, 1204, 1204)
(1, 1204, 1204)
(1, 1204, 1204)
(1, 1204, 1204)
(0, 1204, 1204)
(0, 1204, 1204)
(0, 1204, 1204)
(1, 1204, 1204)
(1, 1204, 1204)
(1, 1204, 1204)
(0, 1204, 1204)
(0, 1204, 1204)
(0, 1204, 1204)


In [ ]:
import napari
from skimage import data

hubble_image = data.hubble_deep_field()

tracks_data = [
    [1, 0, 236, 0],
    [1, 1, 236, 100],
    [1, 2, 236, 200],
    [1, 3, 236, 500],
    [1, 4, 236, 1000],
    [2, 0, 436, 0],
    [2, 1, 436, 100],
    [2, 2, 436, 200],
    [2, 3, 436, 500],
    [2, 4, 436, 1000],
    [3, 0, 636, 0],
    [3, 1, 636, 100],
    [3, 2, 636, 200],
    [3, 3, 636, 500],
    [3, 4, 636, 1000]
]

In [ ]:
hubble_image.shape

In [ ]:
viewer = napari.view_image(hubble_image, name='image')
viewer.add_tracks(tracks_data, name='tracks')
napari.run()